# Analisando o grafo

## Imports

In [ ]:
import netpixi
from netpixi.integration.gt import Graph
from netpixi.integration.gt import *

from regression.integration.gt import *

import seaborn as sns

import cpnet

from graph_tool import spectral
from graph_tool import clustering

import numpy as np

import pandas as pd

import regression as reg

from scipy import stats

from matplotlib import pyplot as plt

import gzip

import json

from datetime import datetime

## Compilando e renderizando o grafo

In [ ]:
g = gt_load('git_hub_sfdp.net.gz')
r = netpixi.render('git_hub_sfdp.net.gz')

## Melhorando visualmento o grafo

In [ ]:
# mudar a cor de fundo do grafo
r.graph(color=0xffffff)

# mudar a cor e o tamanho dos vértices
r.vertex_default(color=0x000000, size=10)

# mudar a cor, a espessura e a curvatura das arestas
r.edge_default(color=0x000000, width=1, curve1=1, curve2=1)

## Análise exploratória do grafo

In [ ]:
# Números de vértices
n = g.num_vertices()

# Número de arestas
m = g.num_edges()

# Número máximo possível de arestas
if g.is_directed():
    max_edges = n * (n - 1)
else:
    max_edges = n * (n - 1) // 2
    
# Densidade
d = m / max_edges


print(f"""Números de vértices: {n}

Número de arestas: {m}

Número máximo possível de arestas: {max_edges}

Densidade: {d}""")

In [ ]:
# Propriedades de cada vértice
data = gt_data(g)

# Degree
degrees = []
for v in g.all_vertices():
    degrees.append(v.total_degree())
    
data['degree'] = degrees

# Histograma do degree
sns.histplot(data['degree']);
data['degree'].describe()

## Análise das variáveis dependentes e independentes

### Criando um novo dataset

In [ ]:
# Dataset refatorado
data_refatorado = pd.read_csv('Github_data_refatorado.csv')

# Dataset do grafo
data_grafo = data_refatorado.copy()

### Coletando os dados

In [ ]:
# Coletando o clustering coefficient de cada vértice

lc = clustering.local_clustering(g)

clustering = [] # Lista de clustering coefficient de cada vértice
for value in lc:
    clustering.append(value)

In [ ]:
# Coletando o nome dos vértices do grafo

nome_arquivo_comprimido = 'git_hub_sfdp.net.gz'
nome_arquivo_descompactado = [] # Lista dos nomes dos vértices do grafo

try:
    with gzip.open(nome_arquivo_comprimido, 'r') as arquivo_comprimido:
        arquivo_comprimido.readline()
        for linha in arquivo_comprimido:
            objeto = json.loads(linha.decode('utf-8'))
            nome_arquivo_descompactado.append(objeto['props']['_id'])
except:
    print("Nome dos vértices coletados")

### Alterando o dataset

In [ ]:
# Filtrando o dataset para somente as os vértices do grafo
data_grafo = data_grafo.loc[data_grafo['name'].isin(nome_arquivo_descompactado)]

# Colocando log nos commits
data_grafo['commits'] = np.log(data_grafo['commits'])

# Colocando log nas stars
data_grafo['star'] = np.log(data_grafo['star'])

# Adicionando clustering coefficient (CC) na tabela
data_grafo['CC'] = clustering

# Adicionando degrees na tabela
data_grafo['degree'] = degrees

# Adicionando pertencimento a comunidade (PC) na tabela
data_grafo['PC'] = data_grafo['degree'] * data_grafo['CC']

# Adicionando days_passed na tabela
def calculate_days_passed(date_string):
    try:
        current_date = datetime.now().date()  # Get the current date
        given_date = datetime.strptime(date_string, '%Y-%m-%d').date()  # Convert the given date string to a date object
        days_passed = (current_date - given_date).days  # Calculate the number of days passed
        return days_passed
    except:
        return None
    
data_grafo['days_passed'] = data_grafo['creation_date'].apply(calculate_days_passed)

In [ ]:
data_grafo.loc[:, ['name', 'star', 'topic_tag', 'commits', 'CC', 'degree', 'PC', 'creation_date', 'days_passed']]

### Resultados

In [ ]:
result = reg.linear(data=data_grafo, formula='commits ~ PC + star')

In [ ]:
result.micro_summary()

In [ ]:
result.nano_summary()

In [ ]:
# Plotando os gráficos de distribuição linear

# Criando subplot
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

# Plotando os gráficos
sns.scatterplot(x=data_grafo["commits"], y=data_grafo["PC"], # Dados
                edgecolor='white', linewidth=0.6, # Personalizando
                ax=axes[0]); # Subplot

sns.regplot(x=data_grafo["commits"], y=data_grafo["PC"], # Dados
            scatter_kws={"edgecolor": "white", "linewidth": 0.6}, # Personalizando
            ax=axes[1]); # Subplot

plt.tight_layout()
plt.show()

In [ ]:
# Plotando o gráfico de resíduos
result.plot_residuals()

## Recortes

### Recorte: PC

In [ ]:
# Mostrando a média dos PCs de cada tópico
print(data_grafo.groupby('topic')['PC'].mean().sort_values(ascending=False))

# Transformando os dados acima em dicionario depois em dataframe para plotagem
topico_PC_dict1 = dict(data_grafo.groupby('topic')['PC'].mean().sort_values(ascending=False))
topico_PC_dict2 = {'Topic': list(topico_PC_dict1.keys()), 'mean_PC': list(topico_PC_dict1.values())}
topico_PC_df = pd.DataFrame(topico_PC_dict2)

# Plotando o gráfico de barras
topico_PC_df.plot.bar(x='Topic', y='mean_PC',
                      legend=False,
                      edgecolor='black',
                      color=['#FFE5A0'] * 3 + ['#EDEDED'] * 12)
plt.xlabel('Tópico')
plt.ylabel('média PC')
plt.show()

In [ ]:
# Copiando o dataset
data_recorte_topicos = data_grafo.copy()

# Fazendo o recorte dos tópicos
data_PC_mean = data_recorte_topicos.groupby('topic')['PC'].mean().reset_index()

# Fazendo o recorte dos tópicos MENOS imersos
recorte_menor = list(data_PC_mean.nsmallest(12, 'PC')['topic']) # Pegando os X menores tópicos
recorte_menor = [i.lower() for i in recorte_menor] # para ficar na mesma formatação dos topic_tag

# Fazendo o recorte dos tópicos MAIS imersos
recorte_maior = list(data_PC_mean.nlargest(3, 'PC')['topic']) # Pegando os X menores tópicos
recorte_maior = [i.lower() for i in recorte_maior] # para ficar na mesma formatação dos topic_tag

print(f"RECORTE_MENOR: {recorte_menor}\nRECORTE_MAIOR: {recorte_maior}")

In [ ]:
# Copiando o dataset
data_recorte = data_grafo.copy()
data_recorte = data_grafo.loc[:, ['topic', 'name', 'star', 'topic_tag', 'commits', 'CC', 'degree', 'PC', 'creation_date', 'days_passed']]

# Recortes do dataset
data_recorte_menores = data_recorte[data_recorte['topic_tag'].apply(lambda x: any(tag in x for tag in recorte_menor))]
data_recorte_maiores = data_recorte[data_recorte['topic_tag'].apply(lambda x: any(tag in x for tag in recorte_maior))]

In [ ]:
# Resultado dos recortes
result_menores = reg.linear(data=data_recorte_menores, formula='commits ~ PC + star')
result_maiores = reg.linear(data=data_recorte_maiores, formula='commits ~ PC + star')

In [ ]:
# Resultado do recorte com os topicos MAIS imersos
result_maiores.nano_summary()

In [ ]:
# Resultado do recorte com os topicos MENOS imersos
result_menores.nano_summary()

### Recorte 2: days_passed

In [ ]:
# Mostrando a média dos days_passed de cada tópico
print(data_grafo.groupby('topic')['days_passed'].mean().sort_values(ascending=False))

# Transformando os dados acima em dicionario depois em dataframe para plotagem
topico_days_passed_dict1 = dict(data_grafo.groupby('topic')['days_passed'].mean().sort_values(ascending=True))
topico_days_passed_dict2 = {'Topic': list(topico_days_passed_dict1.keys()), 'mean_days_passed': list(topico_days_passed_dict1.values())}
topico_days_passed_df = pd.DataFrame(topico_days_passed_dict2)

# Plotando o gráfico de barras
topico_days_passed_df.plot.bar(x='Topic', y='mean_days_passed',
                      legend=False,
                      edgecolor='black',
                      color=['green'] * 2 + ['grey'] * 13)
plt.xlabel('Tópicos')
plt.ylabel('médias days_passed')
plt.show()

In [ ]:
# Copiando o dataset
data_recorte2_topicos = data_grafo.copy()

# Fazendo o recorte dos tópicos
data_days_passed_mean = data_recorte2_topicos.groupby('topic')['days_passed'].mean().reset_index()

# Fazendo o recorte dos tópicos MENOS imersos
recorte2_menor = list(data_days_passed_mean.nsmallest(2, 'days_passed')['topic']) # Pegando os X menores tópicos
recorte2_menor = [i.lower() for i in recorte2_menor] # para ficar na mesma formatação dos topic_tag

# Fazendo o recorte dos tópicos MAIS imersos
recorte2_maior = list(data_days_passed_mean.nlargest(13, 'days_passed')['topic']) # Pegando os X menores tópicos
recorte2_maior = [i.lower() for i in recorte2_maior] # para ficar na mesma formatação dos topic_tag

print(f"RECORTE_MENOR: {recorte2_menor}\nRECORTE_MAIOR: {recorte2_maior}")

In [ ]:
# Copiando o dataset
data_recorte2 = data_grafo.copy()
data_recorte2 = data_grafo.loc[:, ['topic', 'name', 'star', 'topic_tag', 'commits', 'CC', 'degree', 'PC', 'creation_date', 'days_passed']]

# Recortes do dataset
data_recorte2_menores = data_recorte2[data_recorte2['topic_tag'].apply(lambda x: any(tag in x for tag in recorte2_menor))]
data_recorte2_maiores = data_recorte2[data_recorte2['topic_tag'].apply(lambda x: any(tag in x for tag in recorte2_maior))]

In [ ]:
# Resultado dos recortes
result_menores2 = reg.linear(data=data_recorte2_menores, formula='commits ~ PC + star')
result_maiores2 = reg.linear(data=data_recorte2_maiores, formula='commits ~ PC + star')

In [ ]:
# Resultado do recorte com os topicos dos repositórios MAIS antigos
result_maiores2.nano_summary()

In [ ]:
# Resultado do recorte com os topicos dos repositórios MENOS antigos
result_menores2.nano_summary()

### Resultado da união dos recortes

In [ ]:
# SOMENTE Repositórios dos 2 tópicos com os repositórios mais novos + SOMENTE Repositórios com os 3 tópicos mais imersos
r = recorte2_menor + recorte_maior
dr1 = data_grafo.copy()
dr1 = dr1[dr1['topic_tag'].apply(lambda x: any(tag in x for tag in r))]
result_dr1 = reg.linear(data=dr1, formula='commits ~ PC + star')
result_dr1.nano_summary()

In [ ]:
# SOMENTE Repositórios dos 2 tópicos com os repositórios mais novos + TUDO EXCETO Repositórios com os 3 tópicos mais imersos
r = recorte2_menor + recorte_menor
dr1 = data_grafo.copy()
dr1 = dr1[dr1['topic_tag'].apply(lambda x: any(tag in x for tag in r))]
result_dr1 = reg.linear(data=dr1, formula='commits ~ PC + star')
result_dr1.nano_summary()

In [ ]:
# SOMENTE Repositórios dos 2 tópicos com os repositórios mais novos
r = recorte2_menor
dr1 = data_grafo.copy()
dr1 = dr1[dr1['topic_tag'].apply(lambda x: any(tag in x for tag in r))]
result_dr1 = reg.linear(data=dr1, formula='commits ~ PC + star')
result_dr1.nano_summary()

In [ ]:
# TUDO EXCETO Repositórios dos 2 tópicos com os repositórios mais novos + SOMENTE Repositórios com os 3 tópicos mais imersos
r = recorte2_maior + recorte_maior
dr1 = data_grafo.copy()
dr1 = dr1[dr1['topic_tag'].apply(lambda x: any(tag in x for tag in r))]
result_dr1 = reg.linear(data=dr1, formula='commits ~ PC + star')
result_dr1.nano_summary()

In [ ]:
# TUDO EXCETO Repositórios dos 2 tópicos com os repositórios mais novos + TUDO EXCETO Repositórios com os 3 tópicos mais imersos
r = recorte2_maior + recorte_menor
dr1 = data_grafo.copy()
dr1 = dr1[dr1['topic_tag'].apply(lambda x: any(tag in x for tag in r))]
result_dr1 = reg.linear(data=dr1, formula='commits ~ PC + star')
result_dr1.nano_summary()

In [ ]:
# TUDO EXCETO Repositórios dos 2 tópicos com os repositórios mais novos
r = recorte2_maior
dr1 = data_grafo.copy()
dr1 = dr1[dr1['topic_tag'].apply(lambda x: any(tag in x for tag in r))]
result_dr1 = reg.linear(data=dr1, formula='commits ~ PC + star')
result_dr1.nano_summary()

In [ ]:
# SOMENTE Repositórios com os 3 tópicos mais imersos
r = recorte_maior
dr1 = data_grafo.copy()
dr1 = dr1[dr1['topic_tag'].apply(lambda x: any(tag in x for tag in r))]
result_dr1 = reg.linear(data=dr1, formula='commits ~ PC + star')
result_dr1.nano_summary()

In [ ]:
# TUDO EXCETO Repositórios com os 3 tópicos mais imersos
r = recorte_menor
dr1 = data_grafo.copy()
dr1 = dr1[dr1['topic_tag'].apply(lambda x: any(tag in x for tag in r))]
result_dr1 = reg.linear(data=dr1, formula='commits ~ PC + star')
result_dr1.nano_summary()